In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-Del2").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

In [4]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [5]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

# convert star_rating to integer
from pyspark.sql.types import IntegerType
vine_df = vine_df.withColumn("star_rating", vine_df["star_rating"].cast(IntegerType()))

vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [14]:
# Filter data where total_votes > 20
total_votes_filtered_df = vine_df.filter('total_votes>=20')
total_votes_filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|     

In [17]:
# Filter data where helpful_votes/total_votes > 50%
helpful_votes_df = total_votes_filtered_df.filter(total_votes_filtered_df['helpful_votes']/total_votes_filtered_df['total_votes'] >= 0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|     

In [21]:
# Filter data for vine == Y
paid_df = helpful_votes_df.filter(helpful_votes_df['vine'] == 'Y')
paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R32OMS1LSDRCOB|          4|           27|         31|   Y|                N|
|R2IPKK4I0XRCFR|          4|           16|         21|   Y|                N|
| RXYYJ8SGI9IX6|          5|           36|         41|   Y|                N|
|R1SAUBFC7T6JHC|          5|           73|         75|   Y|                N|
|R329HVJKKHWSG3|          5|           59|         59|   Y|                N|
| RZL21K50M52KW|          5|           22|         22|   Y|                N|
|R24RC5J2D92SBE|          5|           19|         20|   Y|                N|
|R1803HITBY7Y08|          4|           92|        104|   Y|                N|
|R1888B8Z6QWYZ4|          4|           18|         25|   Y|                N|
|R1S538NK2I5PRL|          5|           21|         22|   Y|     

In [22]:
# Filter data for vine == N
unpaid_df = helpful_votes_df.filter(helpful_votes_df['vine'] == 'N')
unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|     

In [39]:
# Determine # of reviews, 5-star reviews and 5-star % for paid reviews
paidReviewsCount = paid_df.count()
paidFiveStarCount = paid_df.filter('star_rating=5').count()
paidPercentFiveStar = paidFiveStarCount / paidReviewsCount

print("Number of paid reviews:" , str(paidReviewsCount))
print("Five Star paid reviews:", str(paidFiveStarCount))
print("Percentage of Five Star paid reviews: " + "{:.2%}".format(paidPercentFiveStar))

Number of paid reviews: 334
Five Star paid reviews: 139
Percentage of Five Star paid reviews: 41.62%


In [40]:
# Determine # of reviews, 5-star reviews and 5-star % for unpaid reviews
unpaidReviewsCount = unpaid_df.count()
unpaidFiveStarCount = unpaid_df.filter('star_rating=5').count()
unpaidPercentFiveStar = unpaidFiveStarCount / unpaidReviewsCount

print("Number of unpaid reviews:" , str(unpaidReviewsCount))
print("Five Star unpaid reviews:", str(unpaidFiveStarCount))
print("Percentage of Five Star unpaid reviews: " + "{:.2%}".format(unpaidPercentFiveStar))

Number of unpaid reviews: 61614
Five Star unpaid reviews: 32665
Percentage of Five Star unpaid reviews: 53.02%
